In [ ]:
import numpy as np
import open3d as o3d
#from probreg import cpd
import plotly.graph_objects as go
import pickle
from IPython.display import IFrame
from scipy.spatial.distance import directed_hausdorff
from sklearn.neighbors import NearestNeighbors

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

username ='vatsal0294'
api_key = 'q6t0o4zUgawRXqgWny2d'
tls.set_credentials_file(username=username,api_key=api_key)
# py.plot(fig, filename = f'Sample {index} before reg. RANSACTestoriginal',auto_open=True)


def visualize2clouds(pcd1, pcd2):
    fig = go.Figure(
    data=[
        go.Scatter3d(
            x=pcd1[:,0], y=pcd1[:,1], z=pcd1[:,2], 
            mode='markers',
            name = 'PC',
            marker=dict(size=1, color=None)
        ),
        go.Scatter3d(
            x=pcd2[:,0], y=pcd2[:,1], z=pcd2[:,2], 
            mode='markers',
            name = 'TOF',
            marker=dict(size=1, color=None)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
    )
    return fig

def hausdorffDistance(source,target):
    """
    Computes the Hausdorff distance between two point clouds
    """
    # Compute the Hausdorff distance between the original and augmented point clouds
    hd1 = directed_hausdorff(source, target)[0]
    hd2 = directed_hausdorff(source, target)[0]
    return max(hd1,hd2)

def chamfer_distance(source, target):
    x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric='l2').fit(source)
    y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric='l2').fit(target)
    min_x_to_y = y_nn.kneighbors(source)[0]
    min_y_to_x = x_nn.kneighbors(target)[0]
    chamfer_dist = np.mean(min_x_to_y) + np.mean(min_y_to_x)
    return chamfer_dist

def find_correspondences(source,target,threshold):
    # Create KDTree from the first point cloud
    pcd1_tree = o3d.geometry.KDTreeFlann(target)
    correspondences = []
    for i, point in enumerate(source.points):
        [k, idx, _] = pcd1_tree.search_radius_vector_3d(point, threshold)
        if k > 0:
            correspondences.append([i, idx[0]])  # Use only the first neighbor for simplicity
    return correspondences

def calculate_mse_mae(source,target):
    # Compute distances between the two point clouds
    distances = target.compute_point_cloud_distance(source)
    # Calculate MSE and MAE
    mse = np.mean(np.square(distances))
    mae = np.mean(np.abs(distances))

    return mse,mae

# Loading DIP Result

In [ ]:
with open("DIPResults.pickle", 'rb') as f:
    data = pickle.load(f)
src = data['source']
tgt = data['target']
t_src = data['tsource']

In [ ]:
mean_corrs_before = 0
mean_corrs_after = 0
mse_before = 0
mae_before = 0
mse_after = 0
mae_after = 0
mean_CD = 0
mean_HD = 0
mean_after_CD = 0
mean_after_HD = 0

# Set the distance threshold
threshold = 1.5
for i in range(len(src)):

    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(src[i])
    target = o3d.geometry.PointCloud()
    target.points = o3d.utility.Vector3dVector(tgt[i])
    t_source = o3d.geometry.PointCloud()
    t_source.points = o3d.utility.Vector3dVector(t_src[i])

    correspondences_before = find_correspondences(source,target,threshold)
    correspondences_after = find_correspondences(t_source,target,threshold)

    mean_corrs_before += len(correspondences_before)
    mean_corrs_after += len(correspondences_after)

    mse,mae = calculate_mse_mae(source,target)
    mse_before += mse
    mae_before += mae

    mse,mae = calculate_mse_mae(t_source,target)
    mse_after += mse
    mae_after += mae

    mean_CD += chamfer_distance(src[i],tgt[i])
    mean_after_CD += chamfer_distance(t_src[i],tgt[i])

    mean_HD += hausdorffDistance(src[i],tgt[i])
    mean_after_HD += hausdorffDistance(t_src[i],tgt[i])

print(f'Number of correspondences for search radius {threshold} before registration is {int(mean_corrs_before/len(src))}')
print(f'Number of correspondences for search radius {threshold} after registration is {int(mean_corrs_after/len(src))}')
print(f'Mean square error before registration is {mse_before/len(src)} and Mean absolute error before registration is {mae_before/len(src)}')
print(f'Mean square error after registration is {mse_after/len(src)} and Mean absolute error after registration is {mae_after/len(src)}')

print(f'Mean Chamfer distance before registration is {mean_CD/len(src)}')
print(f'Mean Chamfer distance after registration is {mean_after_CD/len(src)}')
print(f'Mean Hausdorff distance before registration is {mean_HD/len(src)}')
print(f'Mean Hausdorff distance after registration is {mean_after_HD/len(src)}')

# RANSAC Comparison

In [ ]:
with open("../DataPreparation/RANSACData/RANSACTest.pickle", 'rb') as f:
    data = pickle.load(f)
src = data['source']
tgt = data['target']
T = data['transformation']

In [ ]:
mean_corrs_after = 0
mse_after = 0
mae_after = 0
mean_after_CD = 0
mean_after_HD = 0

# Set the distance threshold
threshold = 0.7

for i in range(len(src)):

    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(src[i])
    source.transform(T[i])
    target = o3d.geometry.PointCloud()
    target.points = o3d.utility.Vector3dVector(tgt[i])
    
    correspondences_after = find_correspondences(source,target,threshold)
    mean_corrs_after += len(correspondences_after)

    mse,mae = calculate_mse_mae(source,target)
    mse_after += mse
    mae_after += mae

    mean_after_CD += chamfer_distance(np.array(source.points),np.array(target.points))
    mean_after_HD += hausdorffDistance(np.array(source.points),np.array(target.points))


print(f'Number of correspondences for search radius {threshold} after registration is {int(mean_corrs_after/len(src))}')
print(f'Mean square error after registration is {mse_after/len(src)} and Mean absolute error after registration is {mae_after/len(src)}')
print(f'Mean Chamfer distance after registration is {mean_after_CD/len(src)}')
print(f'Mean Hausdorff distance after registration is {mean_after_HD/len(src)}')

In [ ]:
index = np.random.randint(len(src))
print(index)
index = 106
fig = visualize2clouds(src[index], tgt[index])
fig.show()
py.plot(fig, filename = f'DIP before registration original {index}',auto_open=False)

In [ ]:
fig = visualize2clouds(t_src[index], tgt[index])
fig.show()
#py.plot(fig, filename = f'DIP after registration original {index}',auto_open=False)